In [2]:
import sys
DATA_DIR = '../../../../../Dropbox/Eugenie/data/processed_julian_amazon_data/'

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### Trying to see if the processed data is in good shape
* get did_reviews_mcauley.csv by dropping duplicated rows that were generated when creating the weekly stats
* get did_products_mcauley.csv by dropping duplicated \['asin', 'week'\] pairs

In [6]:
df = pd.read_csv('../../../../../Dropbox/Eugenie/data/processed_julian_amazon_data/merged_Appliances.csv', index_col=0)

In [8]:
df['reviewTime'] = pd.to_datetime(df['reviewTime']).dt.strftime('%Y-%m-%d')

In [18]:
df['week'] = df['reviewTime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') - timedelta(days=datetime.strptime(x, '%Y-%m-%d').weekday()))


In [19]:
groups = df.groupby(['asin','week'], as_index=False)

In [44]:
df = pd.read_csv(DATA_DIR+'mcauley_top10_numeric.csv', index_col=0)

/Users/yujinglai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yujinglai/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [42]:
df.head()

,category,week_n,week_rating,week_word_count,week_helpful_yes,week_image_count,week,after_ban,brand,amazon,price,non_amazon
item_id,,,,,,,,,,,,
B001XXUOQI,Cell Phones & Accessories,1,5.000000,37.000000,0.0,9,2017-02-26,1,Samsung,0,4.99,1
B001XXUOQI,Cell Phones & Accessories,1,5.000000,10.000000,0.0,9,2017-02-19,1,Samsung,0,4.99,1
B001XXUOQI,Cell Phones & Accessories,1,5.000000,11.000000,0.0,9,2017-01-01,1,Samsung,0,4.99,1
B001XXUOQI,Cell Phones & Accessories,1,4.000000,10.000000,0.0,9,2016-12-25,1,Samsung,0,4.99,1
B001XXUOQI,Cell Phones & Accessories,3,4.333333,25.333333,0.0,9,2016-12-11,1,Samsung,0,4.99,1


In [43]:
df.shape

(139504, 12)

In [38]:
df.columns

Index(['vote', 'verified', 'reviewTime', 'reviewerID', 'asin', 'word_count',
       'image', 'category', 'brand', 'price', 'main_cat', 'amazon',
       'total_review_count', 'sim1', 'week', 'avg_rating',
       'weekly_review_count', 'avg_word_count', 'avg_vote', 'avg_image'],
      dtype='object')

In [37]:
df = df.rename(columns={'review_count_x':'total_review_count','review_count_y':'weekly_review_count'})

In [47]:
df.to_csv(DATA_DIR+'mcauley_top10_numeric.csv')

In [46]:
df.drop_duplicates(keep='first',inplace=True)

In [48]:
df.shape

(228762, 20)

### Investigate why the data size is smaller then the previous one (processed with review text)
* Dropped the products with bad data (i.e., css/html files in title attribute)
* Products similar to Amazon products have less reviews than the previous selected ones

In [49]:
df.sim1.isna().sum()

0

In [50]:
df.groupby('category')['brand'].nunique()

category
Automotive                    10
Cell Phones & Accessories     51
Clothing, Shoes & Jewelry     75
Electronics                  508
Home & Kitchen               376
Office Products              182
Patio, Lawn & Garden         101
Pet Supplies                  68
Sports & Outdoors             93
Tools & Home Improvement      38
Name: brand, dtype: int64

In [51]:
df.groupby('category')['asin'].nunique()

category
Automotive                     11
Cell Phones & Accessories      75
Clothing, Shoes & Jewelry     148
Electronics                  1119
Home & Kitchen                775
Office Products               303
Patio, Lawn & Garden          154
Pet Supplies                   87
Sports & Outdoors             125
Tools & Home Improvement       46
Name: asin, dtype: int64

In [52]:
df_amazon = df.loc[df['amazon'] == 1]

In [53]:
df_amazon.groupby('category')['asin'].nunique()

category
Automotive                     1
Cell Phones & Accessories      8
Clothing, Shoes & Jewelry     15
Electronics                  142
Home & Kitchen               126
Office Products               45
Patio, Lawn & Garden          27
Pet Supplies                  12
Sports & Outdoors             13
Tools & Home Improvement       9
Name: asin, dtype: int64

In [54]:
reviews = pd.read_csv(DATA_DIR+'did_reviews_mcauley.csv', index_col=0)

/Users/yujinglai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yujinglai/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [61]:
reviews_amazon = reviews.loc[reviews['amazon'] == 1]

In [62]:
reviews_amazon.groupby('category')['item_id'].nunique()

category
Automotive                     1
Cell Phones & Accessories      9
Clothing, Shoes & Jewelry     16
Electronics                  208
Home & Kitchen               127
Office Products               46
Patio, Lawn & Garden          27
Pet Supplies                  12
Sports & Outdoors             18
Tools & Home Improvement       9
Name: item_id, dtype: int64

In [59]:
reviews = reviews.reset_index()

In [64]:
# Remove asin with less than 10 reviews
reviews['review_count'] = reviews.groupby('item_id')['item_id'].transform('count') # Count reviews/asin
reviews = reviews.loc[reviews['review_count'] > 10]

In [65]:
reviews_amazon = reviews.loc[reviews['amazon'] == 1]
reviews_amazon.groupby('category')['item_id'].nunique()

category
Automotive                     1
Cell Phones & Accessories      9
Clothing, Shoes & Jewelry     16
Electronics                  208
Home & Kitchen               127
Office Products               46
Patio, Lawn & Garden          27
Pet Supplies                  12
Sports & Outdoors             18
Tools & Home Improvement       9
Name: item_id, dtype: int64

Debug

In [28]:
df = pd.read_csv(DATA_DIR+'did/products_mcauley_description.csv', index_col=0)

In [5]:
df.head()

,vote,verified,reviewTime,reviewerID,asin,word_count,image,category,brand,price,main_cat,amazon,total_review_count,sim1,week,avg_rating,weekly_review_count,avg_word_count,avg_vote,avg_image
overall,,,,,,,,,,,,,,,,,,,,
1.0,0.0,True,2016-07-21,A2NJTK1AFAM32F,B0046LTU9G,15,25,Cell Phones & Accessories,Nokia,NaN,Cell Phones & Accessories,0,26,B00MJKZHRK,2016-07-18,1.0,2,64.0,0.0,25.0
1.0,0.0,True,2016-07-07,A3M5VZWB3YV9ES,B0046LTU9G,260,25,Cell Phones & Accessories,Nokia,NaN,Cell Phones & Accessories,0,26,B00MJKZHRK,2016-07-04,1.0,1,260.0,0.0,25.0
1.0,0.0,True,2016-06-28,A2WOZI7YEGBJZ7,B0046LTU9G,220,25,Cell Phones & Accessories,Nokia,NaN,Cell Phones & Accessories,0,26,B00MJKZHRK,2016-06-27,3.0,2,133.5,0.0,25.0
4.0,0.0,True,2016-06-25,A2SYRT7VLSP2J8,B0046LTU9G,22,25,Cell Phones & Accessories,Nokia,NaN,Cell Phones & Accessories,0,26,B00MJKZHRK,2016-06-20,2.5,2,89.0,0.0,25.0
1.0,0.0,True,2016-06-16,AKJWFQ99PPD3X,B0046LTU9G,355,25,Cell Phones & Accessories,Nokia,NaN,Cell Phones & Accessories,0,26,B00MJKZHRK,2016-06-13,1.0,1,355.0,0.0,25.0


In [19]:
def handle_price(x):
    if isinstance(x, str):
        x = x.replace('$','')
        prices = x.replace(' ','').split('-')
        prices = [float(i) for i in prices]
        x = np.mean(prices)
        return x
    return np.NaN

In [27]:
df['price'].isna().sum()

18251

In [29]:
df['price'].isna().sum()

18251

In [26]:
df['price'] = df['price'].apply(handle_price)
df.loc[df['asin'] == 'B013B83VDK']

,vote,verified,reviewTime,reviewerID,asin,word_count,image,category,brand,price,main_cat,amazon,total_review_count,sim1,week,avg_rating,weekly_review_count,avg_word_count,avg_vote,avg_image
overall,,,,,,,,,,,,,,,,,,,,
3.0,0.0,True,2016-04-24,AAFI8TYGODJ8K,B013B83VDK,40,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2016-04-18,3.0,1,40.0,0.0,98.0
5.0,0.0,True,2017-03-19,A1XJG35VO0CZ1W,B013B83VDK,103,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2017-03-13,5.0,1,103.0,0.0,98.0
4.0,4.0,True,2017-01-24,ARWS1QF4YQ0QZ,B013B83VDK,430,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2017-01-23,4.5,2,229.0,2.0,98.0
5.0,0.0,True,2017-01-05,A13OBGNYEFMA78,B013B83VDK,9,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2017-01-02,5.0,1,9.0,0.0,98.0
5.0,0.0,True,2016-12-06,A23MC4PAWL0JAI,B013B83VDK,5,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2016-12-05,5.0,1,5.0,0.0,98.0
5.0,0.0,True,2016-12-01,A1E4RPDK41CV12,B013B83VDK,9,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2016-11-28,5.0,1,9.0,0.0,98.0
5.0,0.0,True,2016-11-23,A378OL4FG7EB0N,B013B83VDK,20,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2016-11-21,5.0,1,20.0,0.0,98.0
5.0,0.0,True,2016-10-29,A10N04RP2JCE5J,B013B83VDK,52,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2016-10-24,5.0,1,52.0,0.0,98.0
5.0,0.0,True,2016-10-05,A353AHSDP5IGPH,B013B83VDK,9,98,Sports & Outdoors,Lookatool,6.67,NaN,0,14,B00R3N036I,2016-10-03,5.0,1,9.0,0.0,98.0


In [25]:
df.loc[df['price'] == '$1.35 - $11.99']

,vote,verified,reviewTime,reviewerID,asin,word_count,image,category,brand,price,main_cat,amazon,total_review_count,sim1,week,avg_rating,weekly_review_count,avg_word_count,avg_vote,avg_image
overall,,,,,,,,,,,,,,,,,,,,
3.0,0.0,True,2016-04-24,AAFI8TYGODJ8K,B013B83VDK,40,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2016-04-18,3.0,1,40.0,0.0,98.0
5.0,0.0,True,2017-03-19,A1XJG35VO0CZ1W,B013B83VDK,103,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2017-03-13,5.0,1,103.0,0.0,98.0
4.0,4.0,True,2017-01-24,ARWS1QF4YQ0QZ,B013B83VDK,430,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2017-01-23,4.5,2,229.0,2.0,98.0
5.0,0.0,True,2017-01-05,A13OBGNYEFMA78,B013B83VDK,9,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2017-01-02,5.0,1,9.0,0.0,98.0
5.0,0.0,True,2016-12-06,A23MC4PAWL0JAI,B013B83VDK,5,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2016-12-05,5.0,1,5.0,0.0,98.0
5.0,0.0,True,2016-12-01,A1E4RPDK41CV12,B013B83VDK,9,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2016-11-28,5.0,1,9.0,0.0,98.0
5.0,0.0,True,2016-11-23,A378OL4FG7EB0N,B013B83VDK,20,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2016-11-21,5.0,1,20.0,0.0,98.0
5.0,0.0,True,2016-10-29,A10N04RP2JCE5J,B013B83VDK,52,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2016-10-24,5.0,1,52.0,0.0,98.0
5.0,0.0,True,2016-10-05,A353AHSDP5IGPH,B013B83VDK,9,98,Sports & Outdoors,Lookatool,$1.35 - $11.99,NaN,0,14,B00R3N036I,2016-10-03,5.0,1,9.0,0.0,98.0
